In [2]:
import serial
import time

class apollo_device():
    def __init__(self, ComPort = 'COM18', ComBaudRate=115200):
        self.port = ComPort
        self.portBaudrate = ComBaudRate

        # Initialize serial connection
        self.ser = serial.Serial(self.port, self.portBaudrate, timeout=0.1)
        
        # Buffer to store received characters
        self.receive_buffer = ""
        
    def send(self, command):
        self.ser.write(command)
        
    def update(self, intensityRed, intensityGreen, intensityBlue, intensityWhite, fan, debugFlag=False):

        # lets send data from here
        char_a = b'A'          # Convert 'A' to bytes
        int_16bit_red = intensityRed     # 16-bit integer value (max 2048)
        int_16bit_green = intensityGreen # 16-bit integer value (max 2048)
        int_16bit_blue = intensityBlue   # 16-bit integer value (max 2048)
        int_16bit_white = intensityWhite # 16-bit integer value (max 2048)
        int_8bit = fan         # 8-bit integer value
        term = b'\n'           # message terminator

        # Convert 16-bit integer to bytes
        bytes_16bit_red = int_16bit_red.to_bytes(2, byteorder='big')
        bytes_16bit_green = int_16bit_green.to_bytes(2, byteorder='big')
        bytes_16bit_blue = int_16bit_blue.to_bytes(2, byteorder='big')
        bytes_16bit_white = int_16bit_white.to_bytes(2, byteorder='big')

        # Convert 8-bit integer to bytes
        bytes_8bit = int_8bit.to_bytes(1, byteorder='big')
        # Combine the bytes to form the complete message
        message = char_a + \
                  bytes_16bit_red + \
                  bytes_16bit_green + \
                  bytes_16bit_blue + \
                  bytes_16bit_white + \
                  bytes_8bit + term
        #message = char_a + bytes_8bit + term
        self.send(message)
        print(bin(message[0]))
        print(bin(message[1]))
        print(bin(message[2]))
        print(bin(message[3]))
        print(bin(message[4]))
        self.read(debug=debugFlag)
        
    def read(self, debug=False):
        #self.ser.reset_input_buffer()
        #time.sleep(0.1)
        retryCount = 50;
        noPackageCount = 0;
        while noPackageCount<retryCount:
            # Read available characters
            data = self.ser.read()

            if data:
                self.receive_buffer = self.receive_buffer + data.decode('utf-8', errors='replace')
                    
                if data == b'\n':
                    if debug:
                        print(self.receive_buffer)
                    self.lastValAsJsonString = "{"+self.receive_buffer+"}"
                    self.receive_buffer = ""
                    noPackageCount = 50;
            else:
                time.sleep(0.01)
                noPackageCount = noPackageCount+1
        
lamp1 = apollo_device()

# try 1
lamp1.update(4,2,2,10,0x88,debugFlag=True)
time.sleep(1)

# try 2
lamp1.update(4,2,2,10,0x88,debugFlag=True)

0b1000001
0b0
0b100
0b0
0b10
0b1000001
0b0
0b100
0b0
0b10


In [2]:
import serial
import time

class apollo_device():
    def __init__(self, ComPort = 'COM16', ComBaudRate=115200):
        self.port = ComPort
        self.portBaudrate = ComBaudRate

        # Initialize serial connection
        self.ser = serial.Serial(self.port, self.portBaudrate, timeout=1)

        # Buffer to store received characters
        self.receive_buffer = ""
        
    def send(self, command):
        self.ser.write(command)
        
    def update(self, intensityRed, intensityGreen, intensityBlue, intensityWhite, fan, debugFlag=False):
        # lets send data from here
        char_a = b'A'          # Convert 'A' to bytes
        int_16bit_red = intensityRed     # 16-bit integer value (max 2048)
        int_16bit_green = intensityGreen # 16-bit integer value (max 2048)
        int_16bit_blue = intensityBlue   # 16-bit integer value (max 2048)
        int_16bit_white = intensityWhite # 16-bit integer value (max 2048)
        int_8bit = fan         # 8-bit integer value
        term1 = b'*'
        term2 = b'\n'           # message terminator

        # Convert 16-bit integer to bytes
        bytes_16bit_red = int_16bit_red.to_bytes(2, byteorder='big')
        bytes_16bit_green = int_16bit_green.to_bytes(2, byteorder='big')
        bytes_16bit_blue = int_16bit_blue.to_bytes(2, byteorder='big')
        bytes_16bit_white = int_16bit_white.to_bytes(2, byteorder='big')

        # Convert 8-bit integer to bytes
        bytes_8bit = int_8bit.to_bytes(1, byteorder='big')
        # Combine the bytes to form the complete message
        message = char_a + \
                  bytes_16bit_red + \
                  bytes_16bit_green + \
                  bytes_16bit_blue + \
                  bytes_16bit_white + \
                  bytes_8bit + \
                  term1 + term2
        #message = char_a + bytes_8bit + term
        print(message)
        
        self.send(message)
        self.ser.reset_input_buffer()
        self.read(debug=debugFlag)
        
    def read(self, debug=False):
        lastChar = ''
        retryCount = 50;
        noPackageCount = 0;
        
        while noPackageCount<retryCount:
            # Read available characters
            data = self.ser.read()
            # If data is not empty, add it to the buffer
            if data:
                self.receive_buffer = self.receive_buffer + data.decode('utf-8', errors='replace')
                if lastChar == b'*' and data == b'\n':
                    messageStart = self.receive_buffer.find('*\"')
                    # remove terminators and scrap from begining of the message
                    fullBuff = self.receive_buffer[messageStart+1:-2]
                    if debug:
                        print(fullBuff)
                    self.lastValAsJsonString = "{"+fullBuff+"}"
                    self.receive_buffer = ""
                    noPackageCount = 50
                else:
                    lastChar = data
            else:
                time.sleep(0.01)
            
    def __del__(self):
        self.ser.close()
        
lamp1 = apollo_device()

In [3]:
# 5600K 208 0.1%
lamp1.update(10,20,10,30,0x88,debugFlag=True)

b'A\x00\n\x00\x14\x00\n\x00\x1e\x88*\n'
A "red_val":0,"green_val":0,"blue_val":0,"white_val":0,"fan_val":0,"temp":0.000000,"redStep":0


In [3]:
lamp1.update(890+5,925+85,205+16,950+30,0x88,debugFlag=True)

b'A\x03\x7f\x03\xf2\x00\xdd\x03\xd4\x88*\n'
"red_val":895,"green_val":1010,"blue_val":221,"white_val":980,"fan_val":136,"temp":0.000000,"redStep":17


In [1]:
25/0.125

200.0

In [3]:
lamp1.lastValAsJsonString

'{\x00\x00\x00\x00\x00�*"red_val":1,"green_val":0,"blue_val":0,"white_val":0,"fan_val":136,"temp":27.500000*\n}'

In [4]:
# 5600K 208 0.1%
lamp1.update(0,2,0,0,0x88,debugFlag=True)

b'A\x00\x00\x00\x02\x00\x00\x00\x00\x88\n'
"red_val":0,"green_val":2,"blue_val":0,"white_val":0,"fan_val":136,"temp":27.750000



In [12]:
lamp1.ser.flush()

In [11]:
a = "*".encode('utf-8')
#b = a.to_bytes(2, byteorder='big')
a

b'*'